In [3]:
import os
import requests

# Define the FastAPI server URL
base_url = "http://localhost:8010"

os.environ["no_proxy"] = "localhost,172.16.87.75"

# 1. Check Server Status
response = requests.get(f"{base_url}/digital_human/check")
if response.status_code == 200:
    print("Server is enabled:", response.json())

# 2. Make a POST request to /digital_human/gen
digital_human_data = {
    "user_id": "user123",
    "request_id": "req123",
    "streamer_id": "1",
    "tts_path": "./data/audio/elon.wav",
    "chunk_id": 0
}

response = requests.post(f"{base_url}/digital_human/gen", json=digital_human_data)
if response.status_code == 200:
    print("Generated digital human video:", response.json())
else:
    print("Failed to generate digital human:", response.text)

# # 3. Make a POST request to /digital_human/preprocess
# preprocess_data = {
#     "user_id": "user123",
#     "request_id": "req123",
#     "streamer_id": "streamer1",
#     "video_path": "/path/to/video.mp4"
# }

# response = requests.post(f"{base_url}/digital_human/preprocess", json=preprocess_data)
# if response.status_code == 200:
#     print("Preprocessing complete:", response.json())
# else:
#     print("Failed to preprocess video:", response.text)


Server is enabled: {'message': 'server enabled'}
Generated digital human video: {'user_id': 'user123', 'request_id': 'req234', 'digital_human_mp4_path': '/space/hotel/phit/personal/rtlipsync/results/avatars/vid_output/req234.mp4'}


In [4]:
import os
import requests

# Define the FastAPI server URL
base_url = "http://localhost:8000"

os.environ["no_proxy"] = "localhost,172.16.87.75"

# 1. Check Server Status
response = requests.get(f"{base_url}/digital_human/check")
if response.status_code == 200:
    print("Server is enabled:", response.json())

# 2. Make a POST request to /digital_human/gen
digital_human_data = {
    "user_id": "user123",
    "request_id": "req345",
    "streamer_id": "1",
    "tts_path": "./data/audio/elon.wav",
    "chunk_id": 0
}

response = requests.post(f"{base_url}/digital_human/gen", json=digital_human_data)
if response.status_code == 200:
    print("Generated digital human video:", response.json())
else:
    print("Failed to generate digital human:", response.text)

# # 3. Make a POST request to /digital_human/preprocess
# preprocess_data = {
#     "user_id": "user123",
#     "request_id": "req123",
#     "streamer_id": "streamer1",
#     "video_path": "/path/to/video.mp4"
# }

# response = requests.post(f"{base_url}/digital_human/preprocess", json=preprocess_data)
# if response.status_code == 200:
#     print("Preprocessing complete:", response.json())
# else:
#     print("Failed to preprocess video:", response.text)


Server is enabled: {'message': 'server enabled'}
Generated digital human video: {'user_id': 'user123', 'request_id': 'req345', 'digital_human_mp4_path': '/space/hotel/phit/personal/rtlipsync/results/avatars/vid_output/req345.mp4'}


In [23]:
# !curl -x "" -X POST http://172.16.87.75:8010/humanaudio -F "file=@data/video/elon.mp4" -F "sessionid=0"

import aiohttp
import asyncio

async def upload_audio_file(session_id, file_path):
    async with aiohttp.ClientSession() as session:
        with open(file_path, 'rb') as file:
            audio_data = {'file': file}
            payload = {'sessionid': str(session_id)}
            form = aiohttp.FormData()
            form.add_field('sessionid', str(session_id))
            form.add_field('file', open(file_path, 'rb'), filename=file_path)
            
            async with session.post('http://localhost:8010/humanaudio', data=form) as response:
                response_json = await response.json()
                print(f"Response: {response_json}")

# Example usage
session_id = 0
file_path = 'data/audio/elon.wav'
asyncio.run(upload_audio_file(session_id, file_path))

{"code":-1,"msg":"err","data":"list index out of range"}


In [21]:
import asyncio
import websockets
import json
import aiohttp
import pyaudio
import av
from aiortc import RTCPeerConnection, RTCSessionDescription, VideoStreamTrack
from aiortc.contrib.media import MediaPlayer
from aiortc import MediaStreamTrack

import os

os.environ["no_proxy"] = "localhost,172.16.87.75,127.0.0.1"


class AudioStreamTrack(MediaStreamTrack):
    kind = "audio"

    def __init__(self, track):
        super().__init__()
        self.track = track

    async def recv(self):
        frame = await self.track.recv()
        return frame


async def send_offer(session, url):
    pc = RTCPeerConnection()

    # Add audio and video transceivers
    pc.addTransceiver('video', direction='sendrecv')
    pc.addTransceiver('audio', direction='sendrecv')

    # Add audio track (replace with actual microphone input or media file)
    audio_track = AudioStreamTrack(MediaPlayer('data/audio/elon.wav').audio)
    pc.addTrack(audio_track)

    # Create an offer and set local description
    offer = await pc.createOffer()
    await pc.setLocalDescription(offer)

    # Wait for ICE gathering to complete
    await ice_gathering_complete(pc)

    # Send the offer to the server
    async with session.post(f'{url}/offer', json={
        'sdp': pc.localDescription.sdp,
        'type': pc.localDescription.type
    }) as response:
        answer = await response.json()

    # Set the remote description
    sessionid = answer.get('sessionid')
    print(f"Session ID: {sessionid}")
    await pc.setRemoteDescription(RTCSessionDescription(sdp=answer['sdp'], type=answer['type']))

    return sessionid, pc


async def send_audio_data(websocket, sessionid, audio_data):
    await websocket.send(json.dumps({
        'type': 'audio',
        'sessionid': sessionid,
        'data': audio_data.tobytes().decode('latin-1')  # Convert bytes to string
    }))


async def receive_video(pc, output_file):
    # Set up a video recorder
    recorder = av.open(output_file, 'w')
    video_stream = recorder.add_stream('h264', rate=30)

    @pc.on("track")
    async def on_track(track):
        if track.kind == "video":
            while True:
                frame = await track.recv()
                # Encode and write the frame
                packet = video_stream.encode(frame)
                if packet:
                    recorder.mux(packet)


async def main():
    url = "http://localhost:8010"  # Replace with your server's address
    ws_url = "ws://localhost:8010/ws"  # WebSocket URL

    async with aiohttp.ClientSession() as session:
        # Establish WebRTC connection
        sessionid, pc = await send_offer(session, url)

        # Set up WebSocket connection
        async with websockets.connect(ws_url) as websocket:
            # Set up PyAudio for microphone input
            p = pyaudio.PyAudio()
            try:
                stream = p.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)
            except IOError as e:
                print(f"Error opening microphone: {e}")
                return

            # Start receiving video
            video_task = asyncio.create_task(receive_video(pc, 'output_video.mp4'))

            try:
                while True:
                    # Read audio data from microphone
                    audio_data = stream.read(1024)
                    # Send audio data through WebSocket
                    await send_audio_data(websocket, sessionid, audio_data)
            except KeyboardInterrupt:
                print("Stopping...")
            finally:
                # Clean up
                stream.stop_stream()
                stream.close()
                p.terminate()
                await pc.close()
                await websocket.close()
                await video_task


def run_async_main():
    try:
        # Check if there's an existing event loop and use it
        loop = asyncio.get_running_loop()
        loop.create_task(main())
    except RuntimeError:
        # If no running loop, use asyncio.run()
        asyncio.run(main())


if __name__ == "__main__":
    run_async_main()


In [20]:
import asyncio
import json
from aiortc import RTCPeerConnection, RTCSessionDescription, MediaStreamTrack
from aiortc.contrib.media import MediaPlayer
from aiohttp import ClientSession
import av  # For video stream handling

pc = None

class AudioStreamTrack(MediaStreamTrack):
    kind = "audio"

    def __init__(self, track):
        super().__init__()  # Don't forget this!
        self.track = track

    async def recv(self):
        frame = await self.track.recv()
        return frame

async def send_offer():
    global pc

    # Add transceivers for video and audio
    pc.addTransceiver('video', direction='recvonly')
    pc.addTransceiver('audio', direction='sendonly')  # Send audio, receive only video

    # Load the audio file to send
    audio_player = MediaPlayer('data/audio/elon.wav')  # Specify your audio file path
    audio_track = AudioStreamTrack(audio_player.audio)
    pc.addTrack(audio_track)

    # Create an offer and set the local description
    offer = await pc.createOffer()
    await pc.setLocalDescription(offer)

    # Wait for ICE gathering to complete
    await ice_gathering_complete()

    # Get the local description after ICE gathering
    offer = pc.localDescription

    # Send the offer to the /offer endpoint on the server
    async with ClientSession() as session:
        async with session.post('http://localhost:8010/offer', json={
            'sdp': offer.sdp,
            'type': offer.type
        }) as response:
            answer = await response.json()

    # Set the remote description with the answer from the server
    sessionid = answer.get('sessionid')
    print(f"Session ID: {sessionid}")
    answer_sdp = RTCSessionDescription(sdp=answer['sdp'], type=answer['type'])
    await pc.setRemoteDescription(answer_sdp)

async def ice_gathering_complete():
    if pc.iceGatheringState == 'complete':
        return

    gathering_complete = asyncio.Future()

    def check_state():
        if pc.iceGatheringState == 'complete':
            pc.removeEventListener('icegatheringstatechange', check_state)
            gathering_complete.set_result(True)

    pc.addEventListener('icegatheringstatechange', check_state)
    await gathering_complete

async def start():
    global pc

    # Configuration for PeerConnection
    config = {
        'sdpSemantics': 'unified-plan'
    }

    # Example of adding STUN servers
    use_stun = False  # Set this flag if you want to use STUN
    if use_stun:
        config['iceServers'] = [{'urls': ['stun:stun.l.google.com:19302']}]

    # Initialize RTCPeerConnection
    pc = RTCPeerConnection(config)

    # Handle incoming video/audio streams
    @pc.on('track')
    async def on_track(track):
        if track.kind == 'video':
            print("Received video track")
            # Handle video stream (e.g., save video stream or display it)
            await handle_video_stream(track, 'output_video.mp4')  # Save video to a file
        elif track.kind == 'audio':
            print("Received audio track")
            # Handle audio stream (e.g., play or process)

    await send_offer()

async def handle_video_stream(track, output_file):
    # Set up a video recorder to save the incoming video stream
    recorder = av.open(output_file, mode='w')
    video_stream = recorder.add_stream('h264', rate=30)

    while True:
        frame = await track.recv()
        # Write each frame to the video stream
        packet = video_stream.encode(frame)
        if packet:
            recorder.mux(packet)

async def stop():
    global pc
    if pc:
        await asyncio.sleep(0.5)
        await pc.close()
        print("Peer connection closed")

def run_async_main():
    try:
        # Check if there's an existing event loop and use it
        loop = asyncio.get_running_loop()
        loop.create_task(start())
    except RuntimeError:
        # If no running loop, use asyncio.run()
        asyncio.run(start())


if __name__ == "__main__":
    run_async_main()